In [17]:
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/home/wyf/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [18]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
from pyml.tree.regression import DecisionTreeRegressor
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.regression import LinearRegression
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.regression import pearson_correlation
from pyml.model_selection import KFold
from pyml.model_selection import ShuffleSplit
from pyml.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 读取数据文件

In [19]:
train = pd.read_excel('../data/train.xlsx')
test = pd.read_excel('../data/testStudent.xlsx')

In [20]:
train.dtypes # 检查有没有数据类型错误的，比如原本是int的变成str，说明里面可能有nan值等奇怪的数据

Additional_Number_of_Scoring                    int64
Average_Score                                 float64
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Tags                                           object
Reviewer_Score                                float64
dtype: object

In [46]:
train_ori_X = train.drop('Reviewer_Score', axis=1).drop('Tags', axis=1)
train_ori_Y = train['Reviewer_Score']
test_ori_X = test.drop('Tags', axis=1)

# 特征工程

In [37]:
def get_proportion_feature_1(df):
    """
    构造以下三个特征
    积极评论占总评论的比例
    消极评论占总评论的比例
    评论员评论占总评论的比例
    """
    df = df.copy()
    
    base_features = ['Total_Number_of_Reviews']
    gap_features = ['Review_Total_Negative_Word_Counts', 'Review_Total_Positive_Word_Counts','Total_Number_of_Reviews_Reviewer_Has_Given']
    for base_feature in base_features:
        for gap_feature in gap_features:
            df[gap_feature+'_radio_'+base_feature] = df[gap_feature]/df[base_feature]
            # 数字太小了，乘上一个10
#             df = df.drop(gap_feature, axis=1)
    return df

# 构造训练集和测试集，并归一化

In [47]:
# 特征方案0：不设置任何特征
train_X_feat = train_ori_X
test_X_feat = test_ori_X

In [38]:
# 特征方案1：增加占比特征，不抛弃原有特征
train_X_feat = get_proportion_feature_1(train_ori_X)
test_X_feat = get_proportion_feature_1(test_ori_X)

In [48]:
train_X_feat.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given'],
      dtype='object')

In [49]:
# 查看不同特征与分数的相关系数
for feat_name in train_X_feat:
    print("{} : {}".format(feat_name, pearson_correlation(train_X_feat[feat_name].values, train_ori_Y.values)))

Additional_Number_of_Scoring : -0.06342829612443453
Average_Score : 0.3622534338191122
Review_Total_Negative_Word_Counts : -0.3823245140424602
Total_Number_of_Reviews : -0.07158845586420134
Review_Total_Positive_Word_Counts : 0.22370965918084207
Total_Number_of_Reviews_Reviewer_Has_Given : 0.002496175494613475


In [50]:
# 归一化，可选择不同方案

In [25]:
# 方案一：没有权重
ss = StandardScaler()
train_X = ss.fit_transform(train_X_feat.values)
test_X = ss.transform(test_X_feat.values)

In [51]:
# 方案二：设置部分列的权重
ss = StandardScaler()
train_X = ss.fit_transform(train_X_feat.values)
test_X = ss.transform(test_X_feat.values)
# 增加某些特征的权重
train_X[:,1] *= 2
train_X[:,4] *= 2
train_X[:,4] *= 2

In [52]:
train_Y = train_ori_Y.values

# 交叉验证

In [56]:
n_splits = 2
cv = ShuffleSplit(n_splits=n_splits)
for train_indices, test_indices in cv.split(train_X):
#     lr = GradientBoostingRegression(learning_rate=0.1, n_estimators=100, max_tree_node_size=400)
    lr = DecisionTreeRegressor(max_node_size=500)
#     lr.fit(train_X[train_indices], train_Y[train_indices], watch=True)
    lr.fit(train_X[train_indices], train_Y[train_indices])
    y_pred = lr.predict(train_X[test_indices])
    print(pearson_correlation(y_pred, train_Y[test_indices]))

0.6396755826209519
0.640723169674117


# 训练模型写入结果

In [54]:
lr = DecisionTreeRegressor(max_node_size=500)
lr.fit(train_X, train_Y)

In [55]:
y_pred = lr.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('./results/'+'CART-m500-weight1-no_feat'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)

# 记录提交结果
## 2018.10.17 第一次rank
1： CART-m500-2018-10-17-10-34.csv : 
    1. 模型：CART二叉回归树
    2. 特征：增加特征1，并抛弃评论词汇数量
    2. 超参数：max_node_size=500,没有设置连续值特征搜索上限 ：
    3. 验证集 0.62左右
    4. 测试集 0.625 左右
    
## 2018.10.17 第二次rank
0： CART-m1000-2018-10-17-10-55 : 
    1. CART二叉回归树
    2. 特征：增加特征1，并抛弃评论词汇数量
    3. 超参数：max_node_size=1000,没有设置连续值特征搜索上限 ：
    3. 验证集 0.605 左右
    4. 测试集 TODO
1: CART-m500-weight12018-10-17-11-23.csv
    1. CART二叉回归树
    2. 特征：增加特征1，保留原有特征
    3. 超参数：
        1. max_node_size=500,没有设置连续值特征搜索上限
        2. 特征权重：
            1. average_score,
            2. Review_Total_Positive_Word_Counts
            3. Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews 在归一化后乘2
    4. 验证集：0.627-0.645
    5. 测试集：TODO
2：CART-m500-weight1-no_feat2018-10-17-11-27.csv
    1. CART二叉回归树
    2. 特征：不修改原有特征
    3. 超参数：
        1. max_node_size=500,没有设置连续值特征搜索上限
        2. 特征权重：
            1. average_score,
            2. Review_Total_Positive_Word_Counts
            3. Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews 在归一化后乘2
    4. 验证集：0.632-0.640
    5. 测试集：TODO